# MTH 652: Advanced Numerical Analysis

## Homework Assignment 3

### <span style="color:red;">Write your name here</span>

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and code.
  * The use of ChatGTP, Copilot, or other AI assistants is **not allowed**
* The code must run in Colab or JupyterHub without errors.
  * Code that does not run will not receive any credit.
  * I suggest double-checking that your code runs properly in a new session. Sometimes code can be broken but appear to work because of old state in the notebook.

### Google Colab Instructions

* After opening this assignment in Google Colab, click on **"Copy to Drive"**
* Rename the notebook to `student_name_mth_652_assignment_3.ipynb`
    * ⚠️ In the above, replace `student_name` with your name!
* Enter your name above (in the cell below "Homework Assignment")!
* When you are ready to submit your assignment, select "File -> Download -> Download .ipynb" from the Colab menu
* Upload the downloaded `.ipynb` file to Canvas

### Assignment Goals

* The purpose of this assignment is to:
    1. Understand and implement the Householder Arnoldi algorithm
    2. Use Householder Arnoldi in GMRES, and compare its convergence to CG

## Written Questions

We will be concerned with the implementation of the **Householder Arnoldi** algorithm.
Consult lecture notes 7 and 8 for details of Arnoldi and Householder.

This is the version of the Arnoldi algorithm (for computing an orthonormal basis for the Krylov subspace) that uses Householder reflections rather than (modified) Gram-Schmidt.

We will fix some notation for what follows.

* Let $A \in \mathbb{R}^{N \times n}$ be an invertible matrix.
* Let the vector $\boldsymbol{u}$ be fixed.
* We will consider the Krylov subspaces $\mathcal{K}_m(A, \boldsymbol{u})$ for $m \leq N$.
    * The matrix whose $i$-th column is the Krylov vector $A^{i-1} \boldsymbol{u}$ is denoted $K_m$.
* We will assume that these spaces are all dimension $m$ (i.e. breakdown doesn't happen).

Let $P_i = I - 2 \boldsymbol{w}_i \boldsymbol{w}_i^T$ denote a Householder reflection, where $\boldsymbol{w}_i$ is a unit vector.

The vectors $\boldsymbol{q}_i$ will denote the orthonormal basis obtained through the Arnoldi process (so that $\operatorname{span}\{ \boldsymbol{q}_i \}_{i=1}^m = \mathcal{K}_m(A,\boldsymbol{u})$).

The vectors $\boldsymbol{e}_i$ are the standard basis vectors (one in the $i$-th place, zero elsewhere).

#### 1. (1 point)

Show that $P_i = P_i^{-1} = P_i^T$.

#### 2. (2 points)

Recall that Householder QR for matrix $B \in \mathbb{R}^{N \times m}$ proceeds by constructing Householder matrices $P_1, \ldots, P_k$ such that
$$
    B_k = P_k P_{k-1} \cdots P_2 P_1 B,
$$
where $B_k$ has zeros below the diagonal in the first $k$ columns.

The end result of this process is the factorization
$$
    B = QR,
$$
where $Q$ is orthogonal and $R$ is upper triangular.
The columns $\boldsymbol{q}_i$ of $Q$ are an orthonormal basis for the column space of $B$.

Show that the first $j$ columns of $P_1 P_2 \cdots P_m$ are the same as those of $P_1 P_2 \cdots P_j$.

From this, conclude that $\boldsymbol{q}_j = P_1 P_2 \cdots P_j \boldsymbol{e}_j$.

#### 3. (2 points)

We apply Householder QR to the Arnoldi iteration.
At the $k$-th iteration, after finding orthonormal basis vectors $\boldsymbol{q}_1, \ldots, \boldsymbol{q}_{k-1}$ that span $\mathcal{K}_{k-1}(A, \boldsymbol{u})$, the vector $A \boldsymbol{q_{k-1}}$ is orthonormalized (using a Householder reflection $P_k$) against the previous vectors to obtain $\boldsymbol{q}_k$.

Recall the key Arnoldi identity
$$
    A Q_{k-1} = Q_k H_{k-1},
$$
where $Q_j$ denotes the $N \times j$ matrix whose $i$-th column is $\boldsymbol{q}_i$, and $H_{k-1}$ is a $k \times (k-1)$ upper Hessenberg matrix.

Show that $\boldsymbol h_{k-1}$ (the last column of $H_{k-1}$) is given by the first $k$ entries of the vector
$$
    P_k P_{k-1} \cdots P_2 P_1 A \boldsymbol{q}_{k-1}.
$$

#### 4. (3 points)

Verify that the following algorithm performs the Householder Arnoldi iteration as described above.

What is the computational complexity in terms of $N$, $m$, and $\mathrm{nnz}(A)$? (Note that the $N \times N$ matrices $P_j$ do **not** need to be formed explicitly).

What are the memory requirements?

**Algorithm (Householder Arnoldi)**

Given $A$, $\boldsymbol{u}$, $m$ as inputs.

Set $\boldsymbol{z} \gets \boldsymbol{u}$

**For** $k = 1, 2, \ldots, m$

* $\boldsymbol{z} \gets P_{k-1} \cdots P_1 \boldsymbol{z}$ (skip this on the first iteration, the product is 'empty')
* $\boldsymbol{z} \gets (z_i)_{i=k}^N$ (last $N - k + 1$ entries of $\boldsymbol{z}$)
* $\boldsymbol{w'} \gets \dfrac{\boldsymbol{z'} + \operatorname{sign}(z'_1)\|\boldsymbol{z'}\|\boldsymbol{e}_1}{\big\|\boldsymbol{z'} + \operatorname{sign}(z'_1)\|\boldsymbol{z'}\|\boldsymbol{e}_1\big\|}$
* $\boldsymbol{w} \gets (0, \boldsymbol{w}') \in \mathbb{R}^N$ (pad with zeros in the first $(k-1)$ entries)
* $P_k \gets I - 2 \boldsymbol{w} \boldsymbol{w'}$
* $\boldsymbol{h}_{k-1} \gets P_k \boldsymbol{z}$
* $\boldsymbol{q}_k \gets P_1 P_2 \cdots P_k \boldsymbol{e_j}$
* $\boldsymbol{z} \gets A \boldsymbol{q}_k$

**End For**

#### 5. (4 points)

Given a vector $(a, b)$, find $\theta$ such that
$$
\begin{pmatrix}
    \cos(\theta) & -\sin(\theta) \\
    \sin(\theta) & \cos(\theta)
\end{pmatrix}
\begin{pmatrix} a \\ b \end{pmatrix}
=
\begin{pmatrix}
\sqrt{a^2 + b^2} \\ 0
\end{pmatrix}.
$$

The matrix
$$
G(\theta)
=
\begin{pmatrix}
    \cos(\theta) & -\sin(\theta) \\
    \sin(\theta) & \cos(\theta)
\end{pmatrix}
$$
is called a **Givens rotation**.

Let $H$ be an upper Hessenberg matrix ($H_{ij} = 0$ for $i > j + 1$).

Explain how a sequence of such Givens rotations can be used to compute the QR factorization of $H$.
What is the connection to the least-squares problem in GMRES?


#### 6. (3 points)

Recall from the lecture notes that $A$ is symmetric, then the upper Hessenberg matrix $H_k$ is **tridiagonal**.
This implies that
$$
A \bm q_m = H_{m-1,m} \bm q_{m-1} + H_{mm} \bm q_m + H_{m+1,m} \bm q_{m+1}.
$$
Constructing an orthonormal basis using this recurrence is known as the **Lanczos** algorithm (rather than Arnoldi).

Suppose a modified version of GMRES is used to solve $A \boldsymbol{x} = \boldsymbol{b}$, where $A$ is known to be symmetric.
Assume further that at each iteration, the least-squares problem can be solved in $\mathcal{O}(1)$ time (by updating the QR factorization using Givens rotations; the same can be done in GMRES, as in the previous question).

Using the Lanczos algorithm rather than Arnoldi, what is the computational complexity per iteration of this modified method?
What are the memory requirements?
Compare to standard GMRES.
What does this imply for restarting?

## Coding Questions

#### 7. (2 points)

Write a function `householder(w, x)` that returns $P \boldsymbol{x}$, where $P = I - 2 \boldsymbol{w} \boldsymbol{w}^T$.

Write a function `householder_ej(w, j)` that returns $P \boldsymbol{e}_j$ (this can be computed directly, without explicitly forming the basis vector $\boldsymbol{e}_j$).

In [ ]:
def householder(w, x):
    pass

def householder_ej(w, j):
    pass

#### 8. (5 points)

Write a function `householder_iter(A, y, ws)` that performs one iteration of the Householder Arnoldi method (see question 4), given the matrix $A$ and vector $\boldsymbol{y} = A \boldsymbol{q}_{k-1}$ (to be orthogonalized to obtain the new vector $\boldsymbol{q}_{k}$).
The argument `ws` is a list of vectors $\boldsymbol{w}_1, \ldots, \boldsymbol{w}_{k-1}$ that define the Householder reflections $P_1, \ldots, P_{k-1}$ computed so far.

The function should compute the new basis vector $\boldsymbol{q}_k$, the Householder reflection vector $\boldsymbol{w}_k$, and the Hessenberg coefficients $\boldsymbol{h}_{k-1}$.

Append the new reflection vector $\boldsymbol{w}_k$ to the list `ws` (so the parameter `ws` is used as both an input and an output).
Return the pair $(\boldsymbol{q}_k, \boldsymbol{h}_{k-1})$.

**Note:** if the vector $\boldsymbol{q}_{N+1}$ is requested (i.e. $N$ previous vectors $\boldsymbol{q}_i$ have already been computed), then $A \boldsymbol{q}_N$ can be written as a combination of the previous basis vectors.
In this case, return the zero vector in place of $\boldsymbol{q}_{N+1}$, and $P_N P_{N-1} \cdots P_1 \boldsymbol{y}$ in place of $\boldsymbol{h}_N$.

To check your work, use `householder_iter` to build the matrices $Q_m$ and $H_m$, and verify that the Arnoldi identity
$$
    A Q_m = Q_{m+1} H_m
$$
holds.
You can also check that the returned vectors $\boldsymbol{q}_i$ must all be orthogonal.

In [ ]:
def householder_iter(A, z, ws):
    pass

#### 9. (3 points)

The provided function `gmres` uses `householder_iter` to implement the GMRES algorithm (a naive version redoing the QR factorization for the least squares problem every step, rather than using Givens rotations).

The parameter `m` is the number of iterations to take before restarting.

Run the test case below and explain why the convergence behavior in the two cases follows from the theory.

In [ ]:
def gmres(A, b, u, m, maxit=200):
    N = A.shape[0]
    tol = 1e-6
    print("It.     Residual")

    M = int(np.ceil(maxit / m))

    for k in range(M):
        r = b - A @ u
        alpha = -np.sign(r[0]) * np.linalg.norm(r)

        Q = np.zeros((N, m+1))
        H = np.zeros((m+1,m))
        ws = []
        Q[:,0] = householder_iter(A, r, ws)[0]

        for i in range(1, m+1):
            z = A @ Q[:,i-1]
            q, h = householder_iter(A, z, ws)
            Q[:,i] = q
            H[:len(h),i-1] = h

            # Solve least-squares problem
            rhs = np.zeros(i+1)
            rhs[0] = alpha
            y, res, _, _ = np.linalg.lstsq(H[:i+1,:i], rhs)

            # Report residual, return if converged
            resnorm = np.sqrt(res[0])
            print(k*m + i - 1, "    ", resnorm)
            if resnorm < tol or i + k*m >= maxit:
                return u + Q[:,:i] @ y

        # Update approximate solution, restart
        u += Q[:,:m] @ y

    return u


In [ ]:
A = np.random.rand(10, 10) + 0.5*np.eye(10)
b = np.random.rand(10)
u = np.zeros(10)

gmres(A, b, u, 10);
gmres(A, b, u, 9);

#### 10. (3 points)

Use the code below to construct the finite element stiffness matrix on an $nx \times nx$ grid.
Solve the system for $nx = 20$ (with a random right-hand side) using GMRES with initial guess of zero.
Try two cases, with restart parameter 10, and restart parameter 50.

Code is included below to solve the system using the conjugate gradient method.

Compare the results and efficiency of the two methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def area(K):
    """
    Returns the area of the triangle defined by K.
    """
    M = np.array([[K[0,0], K[0,1], 1],
                  [K[1,0], K[1,1], 1],
                  [K[2,0], K[2,1], 1]])
    return 0.5 * np.linalg.det(M)

def make_stiffness(V, T, B):
    """
    Assembles the stiffness matrix on the mesh defined by (T, V). Eliminates the
    essential boundary conditions defined by B.
    """
    N = V.shape[0]
    A = np.zeros((N, N))

    for it in range(T.shape[0]):
        K = V[T[it,:],:]
        G1 = np.array([[1, 1, 1],
                       [K[0,0], K[1,0], K[2,0]],
                       [K[0,1], K[1,1], K[2,1]]])
        G2 = np.array([[0,0],[1,0],[0,1]])
        G = np.linalg.solve(G1, G2)
        A_K = area(K) * G @ G.T

        A[np.ix_(T[it,:], T[it,:])] += A_K

    A[B,:] = 0.0
    A[:,B] = 0.0
    for i in B:
        A[i,i] = 1.0

    return A

def square_mesh(nx):
    """
    Generates a triangular Cartesian mesh of the unit square with nx vertices in
    each dimension.

    Returns (V, T, B), where V is are the vertex coordinates, T are the triangle
    indices, and B is a list of boundary vertex indices.
    """
    x = np.linspace(0, 1, nx)
    X, Y = np.meshgrid(x, x)
    V = np.stack((X.ravel(), Y.ravel()), axis=1)

    nt = 2*(nx-1)**2
    T = np.zeros((nt, 3), int)

    for iy in range(nx - 1):
        for ix in range(nx - 1):
            v1 = ix + iy*nx
            v2 = ix + 1 + iy*nx
            v3 = ix + (iy + 1)*nx
            v4 = ix + 1 + (iy + 1)*nx
            T[2*ix + iy*2*(nx-1), :] = [v1, v2, v4]
            T[2*ix + 1 + iy*2*(nx-1), :] = [v1, v4, v3]

    B = []
    for i in range(nx):
        B.append(i)
        B.append(i + nx*(nx - 1))
    for i in range(1, nx - 1):
        B.append(nx*i)
        B.append(nx - 1 + nx*i)

    return V, T, B

In [ ]:
from scipy.sparse.linalg import cg

# The matrix A and right-hand side b should be defined above (using the finite
# element code).

it = 0

def print_res(x):
    global it
    it += 1
    print(it, "    ", np.linalg.norm(b - A@x))

print("It.     Residual")
cg(A, b, rtol=1e-7, callback=print_res);